In [1]:
import pandas as pd
import torch

In [2]:
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-emotion")

model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-emotion")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
C:\Users\User\Desktop\programowanie_web_etc\python_projects\data_projects\venv\lib\site-packages\transformers\models\auto\modeling_auto.py:1564: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [3]:
def get_emotion(text):
  input_ids = tokenizer.encode(text + '</s>', return_tensors='pt')

  output = model.generate(input_ids=input_ids,
               max_length=2)
  
  dec = [tokenizer.decode(ids) for ids in output]
  label = dec[0]
  return label

In [4]:
get_emotion("i feel as if i havent blogged in ages are at least truly blogged i am doing an update cute") # Output: 'joy'

'<pad> joy'

In [5]:
get_emotion("i have a feeling i kinda lost my best friend") # Output: 'sadness'

'<pad> sadness'

In [6]:
df_path = 'dataframes/with_counts/combined_count.csv'

In [7]:
df = pd.read_csv(df_path, index_col = 0)

In [8]:
sample_lyrics = df.sample(5)['Song Lyrics'].str.lower()

In [9]:
import re
emotion_regex = re.compile('<pad>\s(.*)')
emotion_regex.findall('<pad> sadness')[0]

'sadness'

In [10]:
for song in sample_lyrics:
    song_emotion = get_emotion(song)
    print('song:')
    print(song)
    print('===================================')
    print('emotion:')
    print(song_emotion)
    print('===================================')
    print('===================================')

song:
 darling you send me darling you send me darling you send me honest you do honest you do honest you do woahohoh you thrill me darling you thrill me baby you you thrill me honest you do honest you do honest you do  at first i thought it was infatuation but it lasted so long so long now i find myself wanting i want you to marry me please take me home yeah yeah yeah yeah yeah yeah  you you send me darling you send me darling you send me honest you do honest you know ya did ya know ya did ya know ya do know youoohhoo send me you got everything it takes to wed me youoohhoo send me and when you do the things you do you know that send me dooohhoo send me honestly you send me you send me baby you send me you move me do you groove me do whoooo you send me ah yes ah yes ah yes thank you to send me guess im a a sinner make me a longer i i i i i i i just cant stand any it thrill me kill me oooh send me send me baby pretty baby oooh yeahyou might also like
emotion:
<pad> love
song:
 ive got a

Token indices sequence length is longer than the specified maximum sequence length for this model (1706 > 512). Running this sequence through the model will result in indexing errors


song:
 calling all comas prisoner on the loose description the spitting image of me except for the heartshaped hole where the hope runs out  shock me awake and tear me apart pinned like a note in a hospital gown prisoner of sleep deeper down the rabbit hole never to be found again  where are you hiding my love cast off like a stone feelings raw and exposed when im out of control pieces were stolen from me or dare i say given away watching the water give in as i go down the drain i appear missing now  i go missing no longer exist one day i hope im someone youd miss you might also like shock me awake tear me apart pinned like a note in a hospital gown deeper asleep further down the rabbit hole never to be found  its only falling in love because you hit the ground  dancing on wire both ends are on fire cut me loose nowhere to run no more room to pretend wandering along the road in the summer night  i go missing no longer exist one day i hope im someone youd miss  shock me awake tear me ap

In [11]:
def assign_emotion(row):
    lyrics = row['Song Lyrics']
    emotion = get_emotion(lyrics)
    emotion = emotion_regex.findall(emotion)[0]
    return emotion

In [12]:
def apply_emotion(df):
    df['emotion'] = df.apply(
        lambda row: assign_emotion(row),
        axis = 1
    )
    return df

In [13]:
df = apply_emotion(df)

In [14]:
df.sample(5)

,Artist,Song Title,Full Title,Release Date,Year,Month,Day,Pageviews,url,featured_count,...,VBD,VBG,VBN,VBP,VBZ,WDT,WP,WP$,WRB,emotion
687,Marvin Gaye,Got to Give It Up,Got to Give It Up by Marvin Gaye,1977-03-15,1977.0,3.0,15.0,80595,https://genius.com/Marvin-gaye-got-to-give-it-...,0,...,5,5,4,26,14,1,1,1,1,joy
278,Ice Cube,Ghetto Vet,Ghetto Vet by Ice Cube (Ft. Mack 10 & Mr. Shor...,1998-11-17,1998.0,11.0,17.0,59946,https://genius.com/Ice-cube-ghetto-vet-lyrics,2,...,5,5,4,26,14,1,1,1,1,sadness
118,Dua Lipa,Lost in Your Light,Lost in Your Light by Dua Lipa (Ft. Miguel),2017-04-21,2017.0,4.0,21.0,192653,https://genius.com/Dua-lipa-lost-in-your-light...,1,...,5,5,4,26,14,1,1,1,1,joy
666,Curtis Mayfield,Freddie’s Dead (Theme From Superfly),Freddie's Dead (Theme From Superfly) by Curtis...,NaN,1972.0,7.0,NaN,16120,https://genius.com/Curtis-mayfield-freddies-de...,0,...,5,5,4,26,14,1,1,1,1,anger
423,Deep Purple,Perfect Strangers,Perfect Strangers by Deep Purple,1984-10-29,1984.0,10.0,29.0,73721,https://genius.com/Deep-purple-perfect-strange...,0,...,5,5,4,26,14,1,1,1,1,sadness


In [15]:
df.to_csv('dataframes/with_counts/combined_count.csv')